# Comparador de Constitucion Chile - España

Usaremos LangChain para comparar la constitución actual chilena (del 80') con la española.

In [1]:
from functools import partial

from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader

## Setup

Cargamos los documentos en PDF directamente

In [2]:
dl = DirectoryLoader(path="./data", 
                     glob="Constitucion*")

In [3]:
docs = dl.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


Chile

In [4]:
docs[0].page_content[:400]

'CONSTITUCIÓN POLÍTICA DE LA REPÚBLICA\n\nTexto actualizado a octubre  de 2010\n\n2\n\n3\n\nCONSTITUCIÓN POLÍTICA DE LA REPÚBLICA DE CHILE\n\nINDICE\n\nCAPÍTULOS\n\nCAPITULO I  BASES DE LA INSTITUCIONALIDAD\n\nCAPITULO II  NACIONALIDAD Y CIUDADANIA\n\nCAPITULO III  DE LOS DERECHOS Y DEBERES\n\nCONSTITUCIONALES\n\nCAPITULO IV  GOBIERNO\n\nPresidente de la República\n\nMinistros de Estado\n\nArts.   1 al 9\n\nArts.  10 al 18\n\nArt'

España

In [5]:
docs[1].page_content[:400]

'CONSTITUCIÓN\n\nESPAÑOLA\n\nAprobada por Las Cortes en sesiones plenarias del\n\nCongreso de los Diputados y del Senado celebradas\n\nel 31 de octubre de 1978\n\nRatificada por el pueblo español en referéndum de\n\n6 de diciembre de 1978\n\nSancionada por S. M. el Rey ante Las Cortes\n\nel 27 de diciembre de 1978\n\n© Agencia Estatal Boletín Oficial del Estado\n\nNIPO: 007-14-093-3\n\nCatálogo de Publicaciones de la Ad'

Hacemos split en pedazos para mandarlo al embedding...

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1024, 
                                      chunk_overlap=200)
splitted_docs = text_splitter.split_documents(docs)

let's embed using OpenAI Ada02

In [7]:
embeddings = OpenAIEmbeddings(chunk_size=1024)

In [8]:
# db = Chroma.from_documents(texts, embeddings)
db = FAISS.from_documents(splitted_docs, embeddings)

we can save a load back...

In [9]:
db.save_local("./data/constituciones.faiss")

In [10]:
# db = FAISS.load_local("./data/constituciones.faiss", embeddings)

We store everything in the vector database `db`

In [11]:
query = "Cuantos años dura el mandato del presidente"

In [12]:
retriever = db.as_retriever(search_kwargs=dict(k=5))

In [13]:
db.similarity_search(query, k=5)

[Document(page_content='el décimo día después de su proclamación.\n\nEl Presidente elegido conforme a alguno de los incisos precedentes durará en el\n\ncargo hasta completar el período que restaba a quien se reemplace y no podrá postular\n\ncomo candidato a la elección presidencial siguiente.86\n\n83 Frase intercalada por el artículo único, letra c) de la ley de Reforma Constitucional N° 20.354.\n\n84 Inciso modificado por el artículo único, Nº 12 de la ley de Reforma Constitucional Nº 18.825.\n\n85 Inciso modificado por el artículo único, letra d),  de la ley de Reforma Constitucional N° 20.354.\n\n86  Artículo  sustituido  por  el  artículo  1º,  Nº  16  de  la  ley  de  Reforma  Constitucional  Nº  20.050.  Anteriormente  había  sido\n\nmodificado por el artículo único, Nº 13 de la ley de Reforma Constitucional Nº 18.825.\n\n27\n\nArtículo  30.-  El  Presidente  cesará  en  su  cargo  el  mismo  día  en  que  se\n\ncomplete su período y le sucederá el recientemente elegido.', metada

Creamos un chat para recuperar info de la base de datos...

In [15]:
from wandb.integration.langchain import WandbTracer

In [16]:
WandbTracer.init({"project": "langchain_constituciones"})

wandb: Streaming LangChain activity to W&B at https://wandb.ai/capecape/langchain_constituciones/runs/yuataiso
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [17]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(), 
                                           retriever, 
                                           return_source_documents=True)

In [18]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [19]:
result['answer']

'El Presidente de la República durará en el ejercicio de sus funciones por el término de cuatro años y no podrá ser reelegido para el período siguiente, según lo establece el Artículo 25 de la Constitución chilena.'

In [20]:
len(result["source_documents"])

5

In [21]:
result["source_documents"]

[Document(page_content='el décimo día después de su proclamación.\n\nEl Presidente elegido conforme a alguno de los incisos precedentes durará en el\n\ncargo hasta completar el período que restaba a quien se reemplace y no podrá postular\n\ncomo candidato a la elección presidencial siguiente.86\n\n83 Frase intercalada por el artículo único, letra c) de la ley de Reforma Constitucional N° 20.354.\n\n84 Inciso modificado por el artículo único, Nº 12 de la ley de Reforma Constitucional Nº 18.825.\n\n85 Inciso modificado por el artículo único, letra d),  de la ley de Reforma Constitucional N° 20.354.\n\n86  Artículo  sustituido  por  el  artículo  1º,  Nº  16  de  la  ley  de  Reforma  Constitucional  Nº  20.050.  Anteriormente  había  sido\n\nmodificado por el artículo único, Nº 13 de la ley de Reforma Constitucional Nº 18.825.\n\n27\n\nArtículo  30.-  El  Presidente  cesará  en  su  cargo  el  mismo  día  en  que  se\n\ncomplete su período y le sucederá el recientemente elegido.', metada

## Comparemos las constituciones...

In [22]:
chat_history = []
query = "Cual pais tiene un sistema parlamentario?"
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [23]:
result['answer']

'Hay muchos países en todo el mundo que tienen un sistema parlamentario, en el que el poder legislativo es ejercido por un parlamento o asamblea. Algunos ejemplos de países con un sistema parlamentario incluyen el Reino Unido, Canadá, Australia, India, Japón, Italia, Noruega, Suecia, Dinamarca, entre otros.'

In [24]:
result['source_documents'][0]

Document(page_content='los demás en proporción a la población.\n\n3.    La elección se verificará en cada circunscripción atendiendo a\n\ncriterios de representación proporcional.\n\n4.    El Congreso es elegido por cuatro años. El mandato de los\n\nDiputados termina cuatro años después de su elección o el día de la\n\ndisolución de la Cámara.\n\n5.    Son electores y elegibles todos los españoles que estén en\n\npleno uso de sus derechos políticos.\n\nLa ley reconocerá y el Estado facilitará el ejercicio del derecho de\n\nsufragio a los españoles que se encuentren fuera del territorio de\n\nEspaña.\n\nEl mandato\n\nparlamentario\n\nEl Congreso de\n\nlos Diputados\n\nSistema\n\nelectoral\n\nCuatro años de\n\nlegislatura\n\n23\n\n6.    Las elecciones tendrán lugar entre los treinta días y sesenta\n\ndías desde la terminación del mandato. El Congreso electo deberá\n\nser convocado dentro de los veinticinco días siguientes a la celebra-\n\nción de las elecciones.\n\nArtículo 69.\n\n1.    

In [25]:
chat_history = [(query, result["answer"])]
query = "Y chile?"
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [26]:
result['answer']

'Sí, Chile tiene un sistema parlamentario. La Constitución Política de la República establece la existencia del Congreso Nacional, que se compone de la Cámara de Diputados y del Senado, y donde se ejerce el poder legislativo.'

In [27]:
result['source_documents'][0]

Document(page_content='desconcentrada en su caso, de conformidad a la ley.\n\nLos  órganos  del  Estado  promoverán  el  fortalecimiento  de  la  regionalización  del\n\npaís  y  el  desarrollo  equitativo  y  solidario  entre  las  regiones,  provincias  y  comunas  del\n\nterritorio nacional.3\n\nArtículo 4° Chile es una república democrática.\n\nArtículo  5°  La  soberanía  reside  esencialmente  en  la  Nación.  Su  ejercicio  se\n\nrealiza por el pueblo a través del plebiscito y de elecciones periódicas y, también, por las\n\nautoridades que esta Constitución establece. Ningún sector del pueblo ni individuo alguno\n\npuede atribuirse su ejercicio.\n\nEl  ejercicio  de  la  soberanía  reconoce  como  limitación  el  respeto  a  los  derechos\n\nesenciales  que  emanan  de  la  naturaleza  humana.  Es  deber  de  los  órganos  del  Estado\n\nrespetar y promover tales derechos, garantizados por esta Constitución, así como por los\n\ntratados internacionales ratificados por Chile y qu

In [28]:
chat_history = [(query, result["answer"])]
query = "Puedes explicarme la diferencia entre los dos?"
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [29]:
result['answer']

'Este texto no se refiere al sistema parlamentario de Chile, sino al sistema parlamentario de España. Por lo tanto, no puedo responder a su pregunta sobre el sistema parlamentario de Chile con este texto. Lo siento.'

In [30]:
chat_history = [(query, result["answer"])]
query = "En españa el rey tiene algun poder?"
result = qa({"question": query, "chat_history": chat_history})

wandb: WARNING Skipping model saving - unable to safely convert LangChain Model to dictionary due to: 


In [31]:
result['answer']

'Según la Constitución española, el Rey de España tiene una serie de poderes y funciones, incluyendo la capacidad de sancionar y promulgar leyes, nombrar y destituir a los miembros del gobierno, ser informado de los asuntos del Estado, presidir el Consejo de Ministros, ejercer el mando supremo de las Fuerzas Armadas, y ejercer el derecho de gracia. Además, el Rey también cumple una función representativa y ceremonial, es el símbolo de la unidad y permanencia del Estado español, y ejerce la más alta representación del Estado español en las relaciones internacionales.'

In [32]:
result["source_documents"][0]

Document(page_content='pueblo\n\nemanan los poderes del Estado.\n\n3.    La forma política del Estado español es la Monarquía parla-\n\nmentaria.\n\nArtículo 2.\n\nLa Constitución se fundamenta en la indisoluble unidad de la\n\nNación española, patria común e indivisible de todos los españoles, y\n\nreconoce y garantiza el derecho a la autonomía de las nacionalidades\n\ny regiones que la integran y la solidaridad entre todas ellas.\n\nUnidad de la\n\nNación y\n\nderecho a la\n\nautonomía\n\nArtículo 3.\n\n1.    El castellano es la lengua española oficial del Estado. Todos los\n\nespañoles tienen el deber de conocerla y el derecho a usarla.\n\n2.    Las demás lenguas españolas serán también oficiales en las\n\nrespectivas Comunidades Autónomas de acuerdo con sus Estatutos.\n\n3.    La riqueza de las distintas modalidades lingüísticas de España\n\nes un patrimonio cultural que será objeto de especial respeto y pro-\n\ntección.\n\nEl castellano y\n\nlas demás\n\nlenguas\n\nespañolas\n\nAr

In [33]:
WandbTracer.finish()

wandb: Finished uploading data to W&B at https://wandb.ai/capecape/langchain_constituciones/runs/yuataiso
